In [ ]:
#!pip install pandas
#!pip install sqlalchemy
#!pip install psycopg2-binary

In [31]:
import pandas as pd
import numpy as np
from sqlalchemy import event
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.sql import text as sql_text


### Step 1: Information about the NOC version 1.3 update Dec2019
in this step we will uste the information from the official website and the NOC we will use 2 files for this process. in this steps is mandatory to have internet connection in order to download the oficial documents

In [32]:
noc_structure = pd.read_csv("https://www.statcan.gc.ca/eng/statistical-programs/document/noc-cnp-2016-structure-v4-eng.csv")

In [33]:
noc_structure.shape

(690, 5)

In [34]:
noc_elements = pd.read_csv("https://www.statcan.gc.ca/eng/statistical-programs/document/noc-cnp-2016-element-v4-eng.csv")

In [35]:
noc_elements.shape

(43126, 9)

the elements csv file has 9 columns and 43,126 rows and the Stucture file has 5 columns and 690 rows. from the sample exploration, the elements table contains null columns that will be remove before insertion to postgrest


In [36]:
noc_elements = noc_elements.dropna(1)

In [37]:
noc_elements.shape

(43126, 5)

### Step 2: Data Wrangling
In this part we will manage all the data to generated a normalize database in Postgres. 
the main idea is to generate all the necesary Stucture for model Training and Search Engine Support 

In [38]:
noc_structure.columns = ["level","hierarchical_structure","code","title","definition"]

##### GROUP NOC

In [39]:
tierOne = noc_structure[noc_structure.level == 1][["code","title","definition"]]
tierOne.columns = ["group_code","group_title","group_definition"]

##### MAJOR CODE

In [40]:
tierTwo = noc_structure[noc_structure.level == 2][["code","title","definition"]]
diff_row_tier = tierTwo[[len(x) > 2 for x in  tierTwo.code]].copy()

In [41]:
data = []
for i in range(len(diff_row_tier)):
    row = diff_row_tier.iloc[i,:].copy()
    start, end = row.code.split("-")
    for n in np.arange(int(start),int(end)+1):
        d = row.copy()
        d.code = str(n).zfill(2)
        data.append(d)

In [42]:
append_data = pd.DataFrame(data)

In [43]:
append_data

,code,title,definition
9,01,Specialized middle management occupations,This major group comprises specialized middle ...
9,02,Specialized middle management occupations,This major group comprises specialized middle ...
9,03,Specialized middle management occupations,This major group comprises specialized middle ...
9,04,Specialized middle management occupations,This major group comprises specialized middle ...
9,05,Specialized middle management occupations,This major group comprises specialized middle ...
56,07,"Middle management occupations in trades, trans...",This major group comprises middle management o...
56,08,"Middle management occupations in trades, trans...",This major group comprises middle management o...
56,09,"Middle management occupations in trades, trans...",This major group comprises middle management o...


In [44]:
tierTwo = tierTwo[~tierTwo.code.isin(diff_row_tier.code)].append(append_data)

In [45]:
tierTwo.columns = ["major_code","major_title","major_definition"]

##### MINOR CODE

In [46]:
tierThree = noc_structure[noc_structure.level == 3][["code","title","definition"]]
tierThree.columns = ["minor_code","minor_title","minor_definition"]

##### UNIT CODE

In [47]:
tierFour = noc_structure[noc_structure.level == 4][["code","title","definition"]]
tierFour.columns = ["unit_code","unit_title","unit_definition"]

In [48]:
noc_elements.columns = ["level","code","title","type","text_description"]

In [49]:
noc_elements["unit_code"] = noc_elements.code.apply(lambda t: str(t).zfill(4))
noc_elements["minor_code"] = noc_elements.unit_code.apply(lambda t4: t4[:3] )
noc_elements["major_code"] = noc_elements.unit_code.apply(lambda t4: t4[:2] )
noc_elements["group_code"] = noc_elements.unit_code.apply(lambda t4: t4[:1] )
noc_elements["title"] = noc_elements.title.str.lower().str.strip()
noc_elements["text_description"] = noc_elements.text_description.str.lower().str.strip()
noc_elements = noc_elements.query("text_description != 'this group performs some or all of the following duties:'")

In [50]:
noc_elements.sample(5)

,level,code,title,type,text_description,unit_code,minor_code,major_code,group_code
7814,4,2121,Biologists and related scientists,All examples,poultry scientist,2121,212,21,2
8595,4,2146,Aerospace engineers,Main duties,"Design and develop aerospace vehicles, systems...",2146,214,21,2
16491,4,4211,Paralegal and related occupations,All examples,paralegal,4211,421,42,4
3415,4,821,Managers in agriculture,Illustrative example(s),vineyard manager,0821,082,08,0
20386,4,6222,Retail and wholesale buyers,Additional information,Progression to management positions in retail ...,6222,622,62,6


In [51]:
information_type = noc_elements["type"].drop_duplicates().to_list()

In [52]:
type_to_table = {t:"" for t in noc_elements.type.drop_duplicates().to_list()}

In [53]:
type_to_table["Illustrative example(s)"] = "noc_unit_example"
type_to_table["All examples"] = "noc_unit_example"
type_to_table["Exclusion(s)"] = "noc_unit_exclusion"
type_to_table["Main duties"] = "noc_unit_main_duties"
type_to_table["Employment requirements"] = "noc_unit_employment_requirements"
type_to_table["Inclusion(s)"] = "noc_unit_inclusion"
type_to_table["Additional information"] = "noc_unit_aditional_information"


In [54]:
type_to_table

{'Illustrative example(s)': 'noc_unit_example',
 'All examples': 'noc_unit_example',
 'Exclusion(s)': 'noc_unit_exclusion',
 'Main duties': 'noc_unit_main_duties',
 'Employment requirements': 'noc_unit_employment_requirements',
 'Inclusion(s)': 'noc_unit_inclusion',
 'Additional information': 'noc_unit_aditional_information'}

### Step 3: Database Stagging/Ingest

In [55]:
USER = "postgres"
PASS = "pass"
HOST = "192.168.1.5"
PORT = "5432"
SCHEMA = "noc_search_engine"

db_connection_str = "postgresql+psycopg2://{}:{}@{}:{}/{}".format(USER,PASS,HOST,PORT,SCHEMA)

In [56]:
db_connection = create_engine(db_connection_str)

In [57]:
sql_query = "SELECT * FROM information_schema.tables WHERE table_schema = 'public'"

In [58]:
db_schema = pd.read_sql(sql_query,con=db_connection)
db_schema

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,noc_search_engine,public,noc_group,BASE TABLE,None,None,None,None,None,YES,NO,None
1,noc_search_engine,public,noc_major,BASE TABLE,None,None,None,None,None,YES,NO,None
2,noc_search_engine,public,noc_minor,BASE TABLE,None,None,None,None,None,YES,NO,None
3,noc_search_engine,public,noc_unit,BASE TABLE,None,None,None,None,None,YES,NO,None
4,noc_search_engine,public,noc_unit_example,BASE TABLE,None,None,None,None,None,YES,NO,None
5,noc_search_engine,public,noc_unit_inclusion,BASE TABLE,None,None,None,None,None,YES,NO,None
6,noc_search_engine,public,noc_unit_exclusion,BASE TABLE,None,None,None,None,None,YES,NO,None
7,noc_search_engine,public,noc_unit_main_duties,BASE TABLE,None,None,None,None,None,YES,NO,None
8,noc_search_engine,public,noc_unit_employment_requirements,BASE TABLE,None,None,None,None,None,YES,NO,None
9,noc_search_engine,public,noc_unit_aditional_information,BASE TABLE,None,None,None,None,None,YES,NO,None


In [ ]:
for table_name in db_schema.table_name:
    db_connection.execute(sql_text("TRUNCATE TABLE {}".format(table_name)).execution_options(autocommit=True))

In [59]:
#@event.listens_for(db_connection, "before_cursor_execute")
#def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
#    if executemany:
#        cursor.fast_executemany = True

tierOne.to_sql("noc_group", con=db_connection, index=False, if_exists="append", schema="public")
tierTwo.to_sql("noc_major", con=db_connection, index=False, if_exists="append", schema="public")
tierThree.to_sql("noc_minor", con=db_connection, index=False, if_exists="append", schema="public")
tierFour.to_sql("noc_unit", con=db_connection, index=False, if_exists="append", schema="public")

In [60]:
for i in type_to_table:
    table = type_to_table.get(i)
    df = noc_elements[noc_elements.type == i][["unit_code","text_description"]].copy()
    df.to_sql(table, con=db_connection, index=False, if_exists="append", schema="public")